Carregando as bibliotecas

In [ ]:
import json
import pandas as pd
import re
import unicodedata
from datasets import Dataset
from transformers import  Trainer, TrainingArguments, BertTokenizer, BertForQuestionAnswering, pipeline, AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch

data = []

# Carregar o arquivo JSON
with open('trn.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))

# Converter para DataFrame
df = pd.DataFrame(data)

# Exemplo de dados
df.head()

def clean_text(text):
    if isinstance(text, str):  # Verificar se é uma string
        text = re.sub(r'[^\w\s]', '', text)
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        text = text.lower().strip()
    return text

print(f"linhas antes da limpeza:{len(df)}")

df.dropna(subset=['title', 'content'], inplace=True)
df.drop_duplicates(subset=['title', 'content'], inplace=True)
df['title'] = df['title'].apply(clean_text)
df['content'] = df['content'].apply(clean_text)
df = df[df['content'].str.strip() != '']
df = df[df['title'].str.strip() != '']

print(f"linhas após da limpeza:{len(df)}")



In [ ]:
def preprocess_function(examples):
    questions = examples['title']  # Perguntas
    contexts = examples['content']  # Respostas
    
    encodings = tokenizer(questions, contexts, truncation=True, padding=True, max_length=512)
    
    start_positions = []
    end_positions = []

    for i in range(len(examples['content'])):
        response = examples['content'][i]
        
        start_idx = response.find(response)
        end_idx = start_idx + len(response) - 1
        
  
        if start_idx == -1: 
            start_idx = 0
            end_idx = 0
        
        start_positions.append(start_idx)
        end_positions.append(end_idx)
    
    encodings['start_positions'] = start_positions
    encodings['end_positions'] = end_positions    
    
    return encodings


In [ ]:
from transformers import Trainer, TrainingArguments, BertForQuestionAnswering, BertTokenizer

# Carregar o modelo e tokenizer
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

df_to_train = pd.read_json('data_cleaned.json', orient='records', lines=True)

dataset = Dataset.from_pandas(df_to_train.head(1000))

tokenized_dataset = dataset.map(preprocess_function, batched=True)

split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)

train_data = split_dataset['train']
eval_data = split_dataset['test']

training_args = TrainingArguments(
    output_dir='./results_v2',
    eval_strategy='epoch', 
    learning_rate=3e-5,
    per_device_train_batch_size=16, 
    num_train_epochs=4, 
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data
)

# # Treinar o modelo
trainer.train()

trainer.save_model("./results_v2/qa_fine_tuned_model")

tokenizer.save_pretrained("./results_v2/qa_fine_tuned_model")


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Map: 100%|██████████| 1000/1000 [00:00<00:00, 5194.26 examples/s]
                                                
 25%|██▌       | 57/228 [00:30<01:11,  2.41it/s]

{'eval_loss': 2.5493550300598145, 'eval_runtime': 1.0815, 'eval_samples_per_second': 92.466, 'eval_steps_per_second': 12.021, 'epoch': 1.0}


                                                 
 50%|█████     | 114/228 [01:00<00:45,  2.52it/s]

{'eval_loss': 2.276130437850952, 'eval_runtime': 1.0435, 'eval_samples_per_second': 95.834, 'eval_steps_per_second': 12.458, 'epoch': 2.0}


                                                 
 75%|███████▌  | 171/228 [01:30<00:23,  2.48it/s]

{'eval_loss': 2.354689836502075, 'eval_runtime': 1.0571, 'eval_samples_per_second': 94.595, 'eval_steps_per_second': 12.297, 'epoch': 3.0}


                                                 
100%|██████████| 228/228 [02:06<00:00,  1.81it/s]


{'eval_loss': 2.5047550201416016, 'eval_runtime': 1.2864, 'eval_samples_per_second': 77.735, 'eval_steps_per_second': 10.106, 'epoch': 4.0}
{'train_runtime': 126.2656, 'train_samples_per_second': 28.511, 'train_steps_per_second': 1.806, 'train_loss': 2.419877972519189, 'epoch': 4.0}


('./results_v2/qa_fine_tuned_model\\tokenizer_config.json',
 './results_v2/qa_fine_tuned_model\\special_tokens_map.json',
 './results_v2/qa_fine_tuned_model\\vocab.txt',
 './results_v2/qa_fine_tuned_model\\added_tokens.json',
 './results_v2/qa_fine_tuned_model\\tokenizer.json')

In [ ]:
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

model = BertForQuestionAnswering.from_pretrained("./results_v2/qa_fine_tuned_model")  # Caminho onde você salvou o modelo
tokenizer = BertTokenizer.from_pretrained("./results_v2/qa_fine_tuned_model")  # Ou o tokenizer que você usou

model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")  # Caminho onde você salvou o modelo
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")  # Ou o tokenizer que você usou

model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")  # Caminho onde você salvou o modelo
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")  # Ou o tokenizer que você usou

context = "high quality 3 layer ballet tutu 12 inches in length"
question = "girls ballet tutu neon pink" 

question_answerer = pipeline("question-answering" , model=model, tokenizer=tokenizer, device=0)

answer  = question_answerer(question=question, context=context)

print(f"Resposta: {answer}")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


Resposta: {'score': 0.012015399523079395, 'start': 33, 'end': 45, 'answer': '12 inches in'}


Lendo o arquivo